In [ ]:
#환경세팅
#observeID.csv는 관측소ID 데이터
#tem_data는 온도 및 폭염 데이터입니다.
import numpy as np
import pandas as pd
import json
import requests
import branca
import folium
folium.__version__

'0.8.3'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#csv 데이터 읽음 
data1=pd.read_csv('tem_data.csv', encoding='cp949', index_col=0, thousands=',')
data1.head()

,지점,폭염여부(O/X),최고체감온도(°C),최고기온(°C),평균기온(°C),최저기온(°C),평균상대습도(%),폭염특보(O/X),폭염영향예보(단계),열대야(O/X),자외선지수(단계)
일시,,,,,,,,,,,
2021-07-01,북강릉(104),X,28.1,26.4,21.8,18.1,86.8,X,,X,높음
2021-07-01,동해(106),X,27.4,25.2,21.9,19.1,90.0,X,,X,매우높음
2021-07-01,서울(108),X,31.3,31.0,26.3,21.4,70.9,O,관심,X,매우높음
2021-07-01,인천(112),X,29.9,29.3,25.6,21.8,67.9,X,관심,X,매우높음
2021-07-01,원주(114),X,31.0,31.3,25.0,19.0,70.0,X,관심,X,높음


In [ ]:
#데이터 정보
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5022 entries, 2021-07-01 to 2021-07-31
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   지점          5022 non-null   object 
 1   폭염여부(O/X)   5022 non-null   object 
 2   최고체감온도(°C)  4980 non-null   float64
 3   최고기온(°C)    4997 non-null   float64
 4   평균기온(°C)    4993 non-null   float64
 5   최저기온(°C)    4996 non-null   float64
 6   평균상대습도(%)   4932 non-null   float64
 7   폭염특보(O/X)   5022 non-null   object 
 8   폭염영향예보(단계)  5022 non-null   object 
 9   열대야(O/X)    5022 non-null   object 
 10  자외선지수(단계)   5022 non-null   object 
dtypes: float64(5), object(6)
memory usage: 470.8+ KB


In [ ]:
data1.describe()

,최고체감온도(°C),최고기온(°C),평균기온(°C),최저기온(°C),평균상대습도(%)
count,4980.000000,4997.000000,4993.000000,4996.000000,4932.000000
mean,32.325542,31.324114,26.250991,22.269576,79.867153
std,2.305050,3.146937,2.166434,1.923889,8.856532
min,21.500000,18.300000,17.100000,14.300000,48.000000
25%,31.175000,29.600000,24.900000,21.000000,74.000000
50%,32.800000,31.900000,26.500000,22.300000,80.000000
75%,34.000000,33.700000,27.900000,23.500000,86.000000
max,36.900000,38.000000,32.500000,28.800000,100.000000


In [ ]:
data1.columns

Index(['지점', '폭염여부(O/X)', '최고체감온도(°C)', '최고기온(°C)', '평균기온(°C)', '최저기온(°C)',
       '평균상대습도(%)', '폭염특보(O/X)', '폭염영향예보(단계)', '열대야(O/X)', '자외선지수(단계)'],
      dtype='object')

In [ ]:
# 컬럼이름 변경
c = ['지점','폭염여부(0/X)','최고체감온도(°C)', '최고기온(°C)', '최저기온(°C)', '폭염특보(O/X)']
del data1['평균기온(°C)'], data1['평균상대습도(%)'], data1['폭염영향예보(단계)'], data1['열대야(O/X)'], data1['자외선지수(단계)']

In [ ]:
data1.columns = c
#기온차 컬럼추가
data1.insert(6,'기온차(최고-최저)',data1['최고기온(°C)'] - data1['최저기온(°C)'])
data1.columns

Index(['지점', '폭염여부(0/X)', '최고체감온도(°C)', '최고기온(°C)', '최저기온(°C)', '폭염특보(O/X)',
       '기온차(최고-최저)'],
      dtype='object')

In [ ]:
#기온차 컬럼 기온차 큰 순서대로 정렬
data1 = data1.sort_values(by='기온차(최고-최저)', ascending=False)

In [ ]:
#데이터(폭염 및 온도 관련 데이터)
#지점에 표시된 숫자는 관측소ID값
#ex) 북강릉은 관측소ID 104
data1

,지점,폭염여부(0/X),최고체감온도(°C),최고기온(°C),최저기온(°C),폭염특보(O/X),기온차(최고-최저)
일시,,,,,,,
2021-07-01,보성(732),X,32.9,33.0,16.7,X,16.3
2021-07-22,양구(556),O,34.5,36.6,20.5,O,16.1
2021-07-22,인제(211),O,34.0,35.9,19.8,O,16.1
2021-07-23,양구(556),O,35.5,36.7,20.7,O,16.0
2021-07-25,양구(556),O,35.2,36.9,20.9,O,16.0
...,...,...,...,...,...,...,...
2021-07-21,시흥(565),O,35.9,NaN,NaN,O,NaN
2021-07-23,논산(615),O,35.1,NaN,NaN,O,NaN
2021-07-24,논산(615),O,35.3,NaN,NaN,O,NaN


In [ ]:
#폭염특보, 폭염여부가 발효되지 않은 지역을 제외
data1 = data1[data1['폭염특보(O/X)']!='X']
data1 = data1[data1['폭염여부(0/X)']!='X']

#기온차가 10°C 이상 지역
data1 = data1[data1['최고기온(°C)'] - data1['최저기온(°C)']>=10]
data1

,지점,폭염여부(0/X),최고체감온도(°C),최고기온(°C),최저기온(°C),폭염특보(O/X),기온차(최고-최저)
일시,,,,,,,
2021-07-22,양구(556),O,34.5,36.6,20.5,O,16.1
2021-07-22,인제(211),O,34.0,35.9,19.8,O,16.1
2021-07-23,양구(556),O,35.5,36.7,20.7,O,16.0
2021-07-25,양구(556),O,35.2,36.9,20.9,O,16.0
2021-07-28,양구(556),O,33.8,36.3,20.3,O,16.0
...,...,...,...,...,...,...,...
2021-07-23,경산(827),O,33.4,32.9,22.9,O,10.0
2021-07-23,부안(243),O,34.0,33.6,23.6,O,10.0
2021-07-29,세종(239),O,33.9,34.1,24.1,O,10.0


In [ ]:
#관측소ID 값 데이터 
#관측소별 위도 경도 표시
data2=pd.read_csv('observeID.csv', encoding='cp949', index_col=0, thousands=',')
data2.head()

,지점명,지점 번호,위도,경도,해발 고도,주소
번호,,,,,,
1,속초,90.0,38.2509,128.5647,18.1,강원도 고성군 토성면 봉포5길 9 속초고층관측소
2,북춘천,93.0,37.9475,127.7547,95.6,강원도 춘천시 신북읍 장본1길 12 춘천기상대
3,철원,95.0,38.1479,127.3042,154.8,강원도 철원군 갈말읍 명성로179번길 26 철원자동기상관측소
4,독도,96.0,37.2370,131.8700,96.2,경상북도 울릉군 울릉읍 독도이사부길 63
5,동두천,98.0,37.9019,127.0607,112.5,경기도 동두천시 방죽로 16-47 동두천자동기상관측소


In [ ]:
data1

,지점,폭염여부(0/X),최고체감온도(°C),최고기온(°C),최저기온(°C),폭염특보(O/X),기온차(최고-최저)
일시,,,,,,,
2021-07-22,양구(556),O,34.5,36.6,20.5,O,16.1
2021-07-22,인제(211),O,34.0,35.9,19.8,O,16.1
2021-07-23,양구(556),O,35.5,36.7,20.7,O,16.0
2021-07-25,양구(556),O,35.2,36.9,20.9,O,16.0
2021-07-28,양구(556),O,33.8,36.3,20.3,O,16.0
...,...,...,...,...,...,...,...
2021-07-23,경산(827),O,33.4,32.9,22.9,O,10.0
2021-07-23,부안(243),O,34.0,33.6,23.6,O,10.0
2021-07-29,세종(239),O,33.9,34.1,24.1,O,10.0


In [ ]:
data2

,지점명,지점 번호,위도,경도,해발 고도,주소
번호,,,,,,
1,속초,90.0,38.2509,128.5647,18.1,강원도 고성군 토성면 봉포5길 9 속초고층관측소
2,북춘천,93.0,37.9475,127.7547,95.6,강원도 춘천시 신북읍 장본1길 12 춘천기상대
3,철원,95.0,38.1479,127.3042,154.8,강원도 철원군 갈말읍 명성로179번길 26 철원자동기상관측소
4,독도,96.0,37.2370,131.8700,96.2,경상북도 울릉군 울릉읍 독도이사부길 63
5,동두천,98.0,37.9019,127.0607,112.5,경기도 동두천시 방죽로 16-47 동두천자동기상관측소
...,...,...,...,...,...,...
585,사하,950.0,NaN,NaN,NaN,부산광역시 사하구 다대로
586,내장산,951.0,35.5087,126.9073,107.8,전라북도 정읍시 내장호반로 273-17 내장산 자연생태학습장
587,장목,953.0,34.9985,128.6796,33.0,경상남도 거제시 장목면 장목리 360-12


In [ ]:
#관측소ID가 누락된 지역 및 기타 정보 누락지역 NaN 데이터 제거
data2 = data2.dropna()

In [ ]:
data2

,지점명,지점 번호,위도,경도,해발 고도,주소
번호,,,,,,
1,속초,90.0,38.2509,128.5647,18.1,강원도 고성군 토성면 봉포5길 9 속초고층관측소
2,북춘천,93.0,37.9475,127.7547,95.6,강원도 춘천시 신북읍 장본1길 12 춘천기상대
3,철원,95.0,38.1479,127.3042,154.8,강원도 철원군 갈말읍 명성로179번길 26 철원자동기상관측소
4,독도,96.0,37.2370,131.8700,96.2,경상북도 울릉군 울릉읍 독도이사부길 63
5,동두천,98.0,37.9019,127.0607,112.5,경기도 동두천시 방죽로 16-47 동두천자동기상관측소
...,...,...,...,...,...,...
581,북상,946.0,35.7798,127.8183,324.9,경상남도 거창군 북상면 갈계리 송계로 731-18 북상초등학교
582,명사,947.0,34.7220,128.5908,31.0,경상남도 거제시 남부면 저구리 423-3
583,삼장,948.0,35.3106,127.8358,139.4,경상남도 산청군 삼장면 덕교리 618-3


In [ ]:
#지점번호 int로 형변환
data2 = data2.astype({'지점 번호': 'int'})

In [ ]:
#서울(108) -> 서울  + 108) 로 데이터 스플릿('('를 기준으로 스플릿)
data1['지점명'] =  data1.지점.str.split('(').str[0]
data1['지점 번호'] = data1.지점.str.split('(').str[1]
data1

,지점,폭염여부(0/X),최고체감온도(°C),최고기온(°C),최저기온(°C),폭염특보(O/X),기온차(최고-최저),지점명,지점 번호
일시,,,,,,,,,
2021-07-22,양구(556),O,34.5,36.6,20.5,O,16.1,양구,556)
2021-07-22,인제(211),O,34.0,35.9,19.8,O,16.1,인제,211)
2021-07-23,양구(556),O,35.5,36.7,20.7,O,16.0,양구,556)
2021-07-25,양구(556),O,35.2,36.9,20.9,O,16.0,양구,556)
2021-07-28,양구(556),O,33.8,36.3,20.3,O,16.0,양구,556)
...,...,...,...,...,...,...,...,...,...
2021-07-23,경산(827),O,33.4,32.9,22.9,O,10.0,경산,827)
2021-07-23,부안(243),O,34.0,33.6,23.6,O,10.0,부안,243)
2021-07-29,세종(239),O,33.9,34.1,24.1,O,10.0,세종,239)


In [ ]:
#지점번호에 남아있는 108) -> 108 로 특수문자 제거
data1['지점 번호'] = data1['지점 번호'].str.replace(pat=r')', repl=r'', regex=True)

In [ ]:
data1

,지점,폭염여부(0/X),최고체감온도(°C),최고기온(°C),최저기온(°C),폭염특보(O/X),기온차(최고-최저),지점명,지점 번호
일시,,,,,,,,,
2021-07-22,양구(556),O,34.5,36.6,20.5,O,16.1,양구,556
2021-07-22,인제(211),O,34.0,35.9,19.8,O,16.1,인제,211
2021-07-23,양구(556),O,35.5,36.7,20.7,O,16.0,양구,556
2021-07-25,양구(556),O,35.2,36.9,20.9,O,16.0,양구,556
2021-07-28,양구(556),O,33.8,36.3,20.3,O,16.0,양구,556
...,...,...,...,...,...,...,...,...,...
2021-07-23,경산(827),O,33.4,32.9,22.9,O,10.0,경산,827
2021-07-23,부안(243),O,34.0,33.6,23.6,O,10.0,부안,243
2021-07-29,세종(239),O,33.9,34.1,24.1,O,10.0,세종,239


In [ ]:
#가공된 지점번호 int형변환
data1['지점 번호']=data1['지점 번호'].astype(int)

In [ ]:
#혹시모르니 한번더 data2의 지점 번호 int형 변환
data2['지점 번호']=data2['지점 번호'].astype(int)

In [ ]:
#데이터 결합
#데이터 결합시 지점번호를 기준으로 자동으로 결합(PK : 지점번호)
data_merge = pd.merge(data1, data2)

In [ ]:
data_merge

,지점,폭염여부(0/X),최고체감온도(°C),최고기온(°C),최저기온(°C),폭염특보(O/X),기온차(최고-최저),지점명,지점 번호,위도,경도,해발 고도,주소
0,양구(556),O,34.5,36.6,20.5,O,16.1,양구,556,38.0980,127.9853,188.9,강원도 양구군 양구읍 정림리 160-10
1,양구(556),O,35.5,36.7,20.7,O,16.0,양구,556,38.0980,127.9853,188.9,강원도 양구군 양구읍 정림리 160-10
2,양구(556),O,35.2,36.9,20.9,O,16.0,양구,556,38.0980,127.9853,188.9,강원도 양구군 양구읍 정림리 160-10
3,양구(556),O,33.8,36.3,20.3,O,16.0,양구,556,38.0980,127.9853,188.9,강원도 양구군 양구읍 정림리 160-10
4,양구(556),O,35.0,35.7,21.1,O,14.6,양구,556,38.0980,127.9853,188.9,강원도 양구군 양구읍 정림리 160-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292,남해(295),O,33.3,32.6,22.3,O,10.3,남해,295,34.8166,127.9264,43.7,경상남도 남해군 이동면 남해대로 2423 남해자동기상관측소
1293,창원(155),O,33.7,33.6,23.4,O,10.2,창원,155,35.1702,128.5729,37.6,경상남도 창원시 마산합포구 가포순환로 172 창원기상대
1294,고흥(262),O,33.0,32.1,22.0,O,10.1,고흥,262,34.6182,127.2757,53.1,전라남도 고흥군 고흥읍 두원로 130 고흥자동기상관측소
1295,보령(235),O,34.2,33.6,23.5,O,10.1,보령,235,36.3272,126.5574,15.5,충청남도 보령시 대해로 450 보령자동기상관측소


In [ ]:
data_merge.columns

Index(['지점', '폭염여부(0/X)', '최고체감온도(°C)', '최고기온(°C)', '최저기온(°C)', '폭염특보(O/X)',
       '기온차(최고-최저)', '지점명', '지점 번호', '위도', '경도', '해발 고도', '주소'],
      dtype='object')

In [ ]:
#히트맵 환경설정 및 위도 경도 데이터를 각 array변수에 담음
from folium.plugins import HeatMap
 
cityList = []
cityLatitude_val = data_merge['위도'].values
cityLongitude_val = data_merge['경도'].values

In [ ]:
cityLatitude_val

array([38.098 , 38.098 , 38.098 , ..., 34.6182, 36.3272, 34.9905])

In [ ]:
cityLongitude_val

array([127.9853, 127.9853, 127.9853, ..., 127.2757, 126.5574, 128.3309])

In [ ]:
type(cityLatitude_val)

numpy.ndarray

In [ ]:
#cityList append
#모든 데이터동안 반복하면서 위도와 경도를 cityList에 append 
from folium.plugins import HeatMap
 
cityList = []

for i in range(0,len(cityLatitude_val)):
    line = []
    line.append(cityLatitude_val[i])
    line.append(cityLongitude_val[i])
    cityList.append(line)

In [ ]:
cityList

[[38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.098, 127.9853],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [38.0599, 128.1671],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.3779, 128.3946],
 [37.

In [1]:
#folium으로 히트맵 출력
#zoom_start 값을 조절하면 히트맵 사이즈 조정 가능
m = folium.Map(
    location = [36.5053542, 127.7043419],
    zoom_start = 10,
    tiles = 'Stamen Terrain'
)
 
HeatMap(cityList).add_to(m)
m.save('map.html')
m

NameError: ignored